Atividade #11: Indo Além (Hugging Face)

Integrantes: Guilherme Araújo Almeida, João Victor Borges, Vitor Facciolo

Tópico Escolhido Tópico: RAG (Retrieved Augmented Generation)

Ideia: construir uma mini-aplicação em que você:

*   Carrega alguns textos locais (por exemplo, um PDF resumido, ou textos sobre um tema),

*   Transforma em vetores usando um modelo de embeddings da HuggingFace.


In [11]:
!pip install transformers accelerate sentence-transformers faiss-cpu

In [12]:
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import faiss
import numpy as np

In [3]:
documentos = [
    "A Inteligência Artificial é uma área da computação que estuda como criar sistemas capazes de realizar tarefas que normalmente exigem inteligência humana.",
    "RAG, ou Retrieval Augmented Generation, combina modelos de linguagem com uma etapa de busca em documentos para gerar respostas mais fiéis a uma base de conhecimento.",
    "Modelos de linguagem grandes, como LLMs, são redes neurais treinadas em grandes quantidades de texto para prever a próxima palavra em uma sequência.",
    "Na abordagem de RAG, primeiro recuperamos os documentos mais relevantes usando embeddings, depois passamos esse contexto para o modelo de geração de texto."
]

In [4]:
# Modelo de embeddings da HuggingFace
modelo_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Cria os vetores (embeddings) para cada documento
embeddings_docs = modelo_embeddings.encode(documentos, convert_to_numpy=True)

# Normalizar (opcional, mas ajuda na busca)
embeddings_docs = embeddings_docs.astype("float32")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
dim = embeddings_docs.shape[1]  # dimensão dos vetores
index = faiss.IndexFlatL2(dim)  # índice simples de similaridade
index.add(embeddings_docs)      # adiciona documentos ao índice

In [6]:
modelo_llm_nome = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(modelo_llm_nome)
modelo_llm = AutoModelForCausalLM.from_pretrained(modelo_llm_nome)
gerador = pipeline("text-generation", model=modelo_llm, tokenizer=tokenizer, max_new_tokens=200)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [7]:
def buscar_documentos(pergunta, k=2):
    # Cria embedding da pergunta
    pergunta_emb = modelo_embeddings.encode([pergunta], convert_to_numpy=True).astype("float32")

    # Busca os k documentos mais similares
    distancias, indices = index.search(pergunta_emb, k)

    resultados = []
    for i in indices[0]:
        resultados.append(documentos[i])
    return resultados

def responder_com_rag(pergunta, k=2):
    # 1) Recuperar documentos relevantes
    docs_relevantes = buscar_documentos(pergunta, k=k)
    contexto = "\n".join(docs_relevantes)

    # 2) Montar o prompt para o LLM
    prompt = (
        "Você é um assistente que responde com base no contexto fornecido.\n\n"
        f"Contexto:\n{contexto}\n\n"
        f"Pergunta do usuário: {pergunta}\n\n"
        "Resposta:"
    )

    # 3) Gerar a resposta com o modelo
    saida = gerador(prompt)[0]["generated_text"]

    # Opcional: só pegar a parte depois de 'Resposta:'
    if "Resposta:" in saida:
        saida = saida.split("Resposta:")[-1].strip()

    return docs_relevantes, saida

In [ ]:
pergunta = "O que é RAG em Inteligência Artificial?"
docs, resposta = responder_com_rag(pergunta, k=2)

print("=== Documentos recuperados ===")
for i, d in enumerate(docs, 1):
    print(f"[{i}] {d}\n")

print("=== Resposta gerada ===")
print(resposta)

In [ ]:
while True:
    pergunta = input("\nDigite uma pergunta (ou 'sair'): ")
    if pergunta.lower() == "sair":
        break
    docs, resposta = responder_com_rag(pergunta)
    print("\nDocumentos usados:")
    for i, d in enumerate(docs, 1):
        print(f"- [{i}] {d}")
    print("\nResposta:")
    print(resposta)